# Iniciando o projeto com as importação das dependencias


### FASE 1 - CONFIGURAÇÃO


In [1]:
# imports do projeto

import pandas as pd
import gdown as gd
import os
from pathlib import Path

### CONSTANTES

In [2]:
# constanttes
SOURCE_FOLDER = 'source'
DF_FILE_NAME = 'df_fraud_credit'
PARQUET = 'parquet'
CSV = 'csv'

### FUNÇÕES


In [3]:
# função que criar os relatórios de Qualidade de Dados

def montar_estatisticas_dataFrame(originial, tratado):
    total_registros = len(originial)
    total_tratado = len(tratado)
    dados_invalidos = total_registros - total_tratado
    dados_validos = total_registros - dados_invalidos
    percentual_erro = dados_invalidos / total_registros

    # Exibir as métricas
    print(f"📊 Total de registros importados: {total_registros}")
    print(f"✅ Registros válidos: {dados_validos}")
    print(f"❌ Registros com erro: {dados_invalidos}")
    print(f"% Registros com erro: {percentual_erro:.2f}%")

In [4]:
# função para criação de pasta

def criar_pasta(folderName):
    os.makedirs(folderName, exist_ok=True)

In [5]:
# função para montar a string do caminho do arquivo
def makeFolderName(folder, fileName, fileFormat):
    return "./{}/{}.{}".format(folder, fileName, fileFormat)

In [6]:
# função que checa se o arquivo já existe na pasta antes de fazer o download
def fileExistis():
    csvFile = makeFolderName(SOURCE_FOLDER, DF_FILE_NAME, CSV)
    parquetFile = makeFolderName(SOURCE_FOLDER, DF_FILE_NAME, PARQUET)

    file_path = Path(csvFile)
    if not file_path.exists():
        print("Start downloading fileName: {}.csv".format(DF_FILE_NAME))
        url = "https://drive.google.com/uc?export=download&id=1Vumu8jo3P3umuUtBZb6mn7YoVIo4X0ON"
        gd.download(url, csvFile, quiet=False)
        print("Finished download fileName: {}".format(DF_FILE_NAME))
        # conversão do dataFrame do formato csv para parquet para reduzir o tamanho do arquivo
        originalData = pd.read_csv(csvFile)
        print("Start parquetFile converter: {}".format(DF_FILE_NAME))
        originalData.to_parquet(parquetFile, engine="pyarrow", index=False)
        print("Finished parquetFile converter: {}".format(DF_FILE_NAME))
    else:
        # conversão do dataFrame do formato csv para parquet para reduzir o tamanho do arquivo
        print("Start load file {}.csv".format(DF_FILE_NAME))
        originalData = pd.read_csv(csvFile)
        print("Finished load file {}.csv".format(DF_FILE_NAME))
        print("Start parquetFile converter: {}.parquet".format(DF_FILE_NAME))
        originalData.to_parquet(parquetFile, engine="pyarrow", index=False)
        print("Finished parquetFile converter: {}.parquet".format(DF_FILE_NAME))

### FASE 2 - INGESTÃO


In [7]:
# criar a pasta pouso para receber os dados
criar_pasta(SOURCE_FOLDER)

In [ ]:
# download dos dados da origem
fileExistis()


Start downloading fileName: df_fraud_credit.csv


In [ ]:
# carregando o dataFrame a partir do arquivo no formato parquet
parquetFile = makeFolderName(SOURCE_FOLDER, DF_FILE_NAME, PARQUET)
initialData = pd.read_parquet(parquetFile, engine="pyarrow")

### FASE 3 - TRATAMENTO DOS DADOS


In [9]:
zeroFilter = initialData.location_region != "0"
processingData = initialData.loc[zeroFilter]

noneRiskScoreFilter = processingData.risk_score != "none"
processingData = processingData.loc[noneRiskScoreFilter]

noneAmountFilter = processingData.amount != "none"
processingData = processingData.loc[noneAmountFilter]

In [ ]:
montar_estatisticas_dataFrame(initialData, processingData)

In [12]:
processingData["risk_score"] = pd.to_numeric(processingData["risk_score"])

In [13]:
riskScoreTable = (
    processingData.groupby("location_region")["risk_score"]
    .mean()
    .reset_index()
    .sort_values(by="risk_score", ascending=False)
)

In [ ]:
print(riskScoreTable.head())

In [15]:
# converter todos os dados da coluna timestamp para o formato dateTime
processingData["timestamp"] = pd.to_datetime(processingData["timestamp"])

# converter todos os dados da coluna amount para o formato Float64
processingData["amount"] = processingData["amount"].values.astype("float64")

In [16]:
salesData = processingData[processingData["transaction_type"] == "sale"]

In [ ]:
salesData.head(n=50)

In [21]:
recentTransactions = salesData.sort_values(
    by=["receiving_address", "timestamp"], ascending=[True, False]
)
recentTransactions = recentTransactions.drop_duplicates(subset=["receiving_address"], keep="first")


In [22]:
topRecentTransactions = recentTransactions.nlargest(3, "amount")[
    ["receiving_address", "amount", "timestamp"]
]

In [ ]:
print("\n📌 Top 3 'receiving address' com maior 'amount':")
print(topRecentTransactions)